# 🎯 學習引導（Learning Guide）

## 📖 本章學完你將能學會什麼：
- 了解 **語音轉文字（Speech to Text, STT）** 的原理與應用場景，如字幕生成、會議記錄、語音助理。  
- 熟悉 OpenAI 的 **Whisper 模型** 架構與使用方法，能正確設定 `model`、`language`、`prompt` 等參數。  
- 能夠使用 Python 與 OpenAI API 進行 **音訊上傳與批次轉錄（Batch Transcription）**。  
- 學會如何透過 **串流模式（Streaming Mode）** 即時獲取轉錄結果。  
- 了解 Whisper 模型在不同任務（Transcribe / Translate）下的差異與適用情境。  
- 理解 **Prompt 對語音辨識準確度的影響**，並能設計上下文提示（Context Prompt）。  
- 熟悉 **文字轉語音（Text to Speech, TTS）** 的應用邏輯與聲音控制要素（語氣、口音、語速、音色）。  
- 掌握 Python 中 **TTS API 的串接與音檔播放方法**（含 pydub、pygame、pyaudio 實作）。  
- 能實作出一個「**語音自助點飲料系統**」，串接 STT → LLM → TTS 的完整流程。

---

## 📘 最終你將具備的能力：
- 💡 **實作能力**：能夠開發出能錄音、轉錄、語音回覆的 Python 系統。  
- 🧠 **理解力**：能清楚解釋 STT 與 TTS 的技術差異與模型特性。  
- 🔧 **整合力**：可將 Whisper、LangChain、TTS 等 API 串成完整應用流程。  
- 🗣️ **應用力**：能設計各種語音互動應用（如客服助理、有聲書、自助點餐系統）。  
- 🚀 **延伸力**：能思考如何將語音模型與其他 AI 模組（RAG、Agent、Ollama）結合，打造更進階的智慧應用。 


# 語音轉文字 (Speech to Text)

在許多應用中，語音資料需要被轉換為文字，例如字幕產生、語音助理、會議紀錄或多語言翻譯。OpenAI 提供了 **Whisper** 模型，可以高效地進行以下兩種主要任務：

- **轉錄 (Transcribe)：** 將音訊檔案轉換成相同語言的文字。  
- **翻譯轉錄 (Translate & Transcribe)：** 將音訊檔案的內容轉錄並翻譯成英文。  

目前檔案上傳限制為 **25 MB**，支援的音訊檔案格式包括：  
`mp3`, `mp4`, `mpeg`, `mpga`, `m4a`, `wav`, `webm`

---

# 文字轉語音 (Text to Speech, TTS)

除了語音轉文字，OpenAI 也提供了 **文字轉語音 (TTS)** 功能。透過 TTS，我們可以將輸入的文字轉換為自然流暢的語音，適合應用於：  

- **語音助理**：讓機器能以自然語音回覆使用者。  
- **內容創作**：自動生成旁白或有聲讀物。  
- **輔助功能**：幫助視覺障礙者或閱讀困難者更容易獲取資訊。  

TTS 支援多種語音風格與音質選擇，能夠在不同應用場景中提供更自然的使用體驗。

---

接下來，我們將透過程式範例來展示如何使用 Python 串接 **Whisper** (語音轉文字) 與 **TTS** (文字轉語音)，並逐步完成音訊與文字的雙向轉換。


## Audio models

| Model   | Usage                                            |
|---------|--------------------------------------------------|
| Whisper |  \$ 0.006 / minute rounded to the nearest second     |
| TTS     |  \$ 15.00 / 1M characters                          |
| TTS HD  |  \$ 30.00 / 1M characters                          |


Whisper 按**分鐘**計費；TTS 按**字元**計費。



In [ ]:
import os

os.chdir("../../")

In [ ]:
from initialization import credential_init

credential_init()

In [ ]:
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

# Transcription

## Batch Transcription

在 非串流模式（使用 transcriptions.create 上傳檔案）時，Whisper 會等待整個音訊檔案完全上傳後，才會對完整音檔進行轉錄，並在完成後一次性回傳結果。

In [ ]:
audio_file= open("tutorial/week_6/孩子上網，小心上當.mp3", "rb")

In [ ]:
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file
)

In [ ]:
transcription.text

## Prompt 參數

>- 給予Context
>- 給予關鍵字，像是私line，應該只有台灣人用這個詞，別太指望OpenAI會知道這種用法

In [ ]:
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  prompt="私line"
)

transcription.text

修正是有用，但沒有scalability。因為這表示你可能需要

- 1.先聽過所有內容，然後用人工校正
- 2.把所有可能有問題的字建立好一個數據庫，然後送入prompt。

Prompt 的最佳使用時機是提供 上下文 (Context) 或 特定領域詞彙 (Domain-specific terminology)

- 例如：「此錄音是關於一個 AI 會議的，請注意其中的專有名詞：GPT-4o, RAG, LangChain。」這樣能更有效利用 Prompt 的價值。

### gpt-4o-mini-transcribe

- 在gpt-4o-transcript 和 gpt-4o-mini-transcript 模型記得加上語言編碼，不然中文輸出極大概率是簡體中文。

In [ ]:
transcription = client.audio.transcriptions.create(
  model="gpt-4o-mini-transcribe", 
  file=audio_file,
  prompt="私line",
  language='zh-tw',
)

transcription.text

In [ ]:
transcription = client.audio.transcriptions.create(
  model="gpt-4o-transcribe", 
  file=audio_file,
  prompt="私line",
  language='zh-tw',
)

transcription.text

假設你需要時間軸

In [ ]:
transcription = client.audio.transcriptions.create(
  model="whisper-1", 
  file=audio_file,
  response_format='srt'
)

In [ ]:
print(transcription)

### Stream

如果你在呼叫時使用 stream=True，SDK 仍然會先上傳整個音訊檔（它並不是一段一段分塊上傳），但不同的是，它不會等到完整的轉錄結束才回傳，而是會在 Whisper 生成的同時，逐步傳回結果。

因此，你會在轉錄過程中陸續看到部分文字片段。

等到 Whisper 完成後，你就會得到最終的完整轉錄結果。

- 只支援 gpt-4o-mini 和 gpt-4o transcription

In [ ]:
audio_file= open("tutorial/week_6/孩子上網，小心上當.mp3", "rb")

stream = client.audio.transcriptions.create(
  file=audio_file,
  model="gpt-4o-mini-transcribe",
  stream=True,
  language='zh-tw',
  prompt='私line',
  temperature=0
)

buffer = ""
for event in stream:
    # Depending on the event format; e.g. maybe:
    # - event.type might be “transcript.text.delta”
    # - or event.delta or event.partial
    try:
        # Example if event has attribute 'delta'
        partial = event.delta
        buffer += partial
        print(partial, end="", flush=True)
    except AttributeError:
        # fallback / final result
        print("\r" + " " * len(buffer), end="")  # 清掉這行
        print("\r" + event.text)  # 印最終結果

簡單的比較gpt-4o-mini-transcribe和gpt-4o-transcribe的差異

In [ ]:
stream = client.audio.transcriptions.create(
  file=audio_file,
  model="gpt-4o-transcribe",
  stream=True,
  language='zh-tw',
  prompt='私line',
  temperature=0
)

buffer = ""
for event in stream:    
    try:
        partial = event.delta
        buffer += partial
        print(partial, end="", flush=True)
    except AttributeError:
        print("\r" + " " * len(buffer), end="")  # 清掉這行
        print("\r" + event.text)  # 印最終結果

gpt-4o-transcribe在表現上比gpt-4o-mini好一些。但要實捶仍然需要大量的測試。
但是大致上應該符合下面的表現:

>- gpt-4o-transcribe (更高品質，更高成本)
>- gpt-4o-mini-transcribe (速度更快，成本更低，但品質略遜)

# 閩南語語音辨識（ASR）可行性評估

## 
1. **書寫標準不統一是瓶頸**  
   - 語音模型訓練需要「語音 + 對應文字轉錄」資料。  
   - 閩南語存在多種書寫系統（漢字、台羅文、白話字/羅馬拼音），缺乏一致標準會造成資料集混亂，難以收斂。  

2. **資源不足**  
   - 相比英語、中文，閩南語的「語音–文字平行語料」極少。  
   - 缺乏大量公開 dataset，從零開始訓練成本極高。  

3. **實用性受限**  
   - 沒有公認的書寫方式，ASR 輸出的結果難以被廣泛採用。  
   - 例如：同一句話輸出為「漢字版」或「台羅文版」，使用者群體可能互不接受。  

---

## 💡 技術上值得補充的觀點
1. **語音模型本身不依賴「官方標準」**  
   - 模型只需要「統一的訓練標籤」。  
   - 無論是台羅文、漢字或羅馬拼音，只要 dataset 標註一致，模型即可學習。  
   - 困難點在於「社群能否就某種書寫系統達成共識」。  

2. **可採「多輸出 / 後處理」策略**  
   - 訓練時使用 **台羅文**（與音素對應性較佳）。  
   - 應用層可透過轉換器將台羅文輸出轉換成漢字或其他拼音系統。  
   - 這樣可繞過「標準未定」的問題。  

3. **現代語音技術降低門檻**  
   - Whisper 等大模型已證明：低資源語言只要有數百小時資料即可微調出可用系統。  
   - 技術上「可行」，只是 **成本高 + 資料缺乏**。  

4. **平行案例**  
   - 客家話、藏語、威爾斯語、愛爾蘭語等語言也有「語料少、書寫多樣」的挑戰。  
   - 仍有人成功建立 ASR → 證明並非「技術上不可能」。  

---

## 📌 綜合評估
- **正確：** 在台灣現況下，缺乏書寫標準確實限制模型實用性與推廣。  
- **補充：** 從純技術角度，書寫標準不是必要條件，只要 dataset 標註一致即可建模。  
- **真正挑戰：**  
  1. 語料不足（收集成本高）  
  2. 缺乏社群共識（採哪一種文字標準）  
  3. 下游應用有限（市場需求小）  

➡️ **結論：** 技術上可行，但現實條件下效益有限。


### 試試看從電腦透過麥克風進行錄音然後使用 Transcribe

- https://www.gyan.dev/ffmpeg/builds/

錄音

In [ ]:
!pip install sounddevice

In [ ]:
import io

import numpy as np
import sounddevice as sd
from pydub import AudioSegment

DURATION = 5  # seconds
FS = 44100    # sample rate

print("Recording...")

# Record audio
audio = sd.rec(int(DURATION * FS), samplerate=FS, channels=1, dtype='int16')
sd.wait()  # Wait until recording is finished
print("Recording finished.")

將聲音數據從 np.array 轉換成 mp3

In [ ]:
# Convert numpy array to AudioSegment
audio_bytes = audio.tobytes()

audio_segment = AudioSegment(
    data=audio_bytes,
    sample_width=audio.dtype.itemsize,
    frame_rate=FS,
    channels=1
)

# Save as MP3 in-memory (as an object)
mp3_io = io.BytesIO()
audio_segment.export(mp3_io, format="mp3")
mp3_io.seek(0)  # Rewind to start

# https://community.openai.com/t/openai-whisper-send-bytes-python-instead-of-filename/84786/3

mp3_io.name = "word.mp3"

In [ ]:
# 將上述步驟寫成一函數

def audio_to_mp3_io(audio, FS:int, channels: int):

    # Convert numpy array to AudioSegment
    audio_bytes = audio.tobytes()
    
    audio_segment = AudioSegment(
        data=audio_bytes,
        sample_width=audio.dtype.itemsize,
        frame_rate=FS,
        channels=channels
    )
    
    # Save as MP3 in-memory (as an object)
    mp3_io = io.BytesIO()
    audio_segment.export(mp3_io, format="mp3")
    mp3_io.seek(0)  # Rewind to start

    mp3_io.name = "word.mp3"

    return mp3_io

在給transcription的endpoint中，file的部分不是給檔案名稱，而是直接將檔案(在記憶體中)直接傳給API。

In [ ]:
stream = client.audio.transcriptions.create(
    model="gpt-4o-transcribe", 
    file=mp3_io,
    response_format="text",
    language='zh-tw',
    stream=True
)

buffer = ""
for event in stream:
    # Depending on the event format; e.g. maybe:
    # - event.type might be “transcript.text.delta”
    # - or event.delta or event.partial
    try:
        # Example if event has attribute 'delta'
        partial = event.delta
        buffer += partial
        print("\r" + buffer, end="", flush=True)
        # print(partial, end="", flush=True)
    except AttributeError:
        # fallback / final result
        print("\r" + " " * len(buffer), end="")  # 清掉這行
        print("\r" + event.text)  # 印最終結果

在先前的範例中，我們沒辦法決定從哪個時候開始錄音

現在我們加入起始和結束的控制

In [ ]:
import threading

FS = 44100
CHANNELS = 1
dtype = 'int16'

recorded_frames = []

def callback(indata, frames, time, status):
    recorded_frames.append(indata.copy())

def record_audio():
    # 加入起始和結束的控制
    with sd.InputStream(samplerate=FS, channels=CHANNELS, 
                        dtype=dtype, callback=callback):
        input("Press Enter to start recording...")
        print("Recording... Press Enter again to stop.")
        input()
        print("Recording stopped.")


In [ ]:
# Clear frames before each recording
recorded_frames.clear()
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()
recording_thread.join()

為何要使用threading而不是直接執行record_audio?

- 直接執行record_audio會堵塞整個程式
- 其他的代碼，像是UI update, logging, 背景處理等等都無法做任何事

一個小型示範:

In [ ]:
recorded_frames.clear()
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()

print("同時在做其他的事情")

recording_thread.join()

In [ ]:
# Combine recorded frames
if recorded_frames:
    audio_np = np.concatenate(recorded_frames, axis=0)
else:
    audio_np = np.array([], dtype=dtype)

mp3_io =  audio_to_mp3_io(audio=audio_np, FS=FS, channels=1)

In [ ]:
stream = client.audio.transcriptions.create(
    model="gpt-4o-transcribe", 
    file=mp3_io,
    response_format="text",
    language='zh-tw',
    stream=True
)

buffer = ""
for event in stream:
    # Depending on the event format; e.g. maybe:
    # - event.type might be “transcript.text.delta”
    # - or event.delta or event.partial
    try:
        # Example if event has attribute 'delta'
        partial = event.delta
        buffer += partial
        print("\r" + buffer, end="", flush=True)
        # print(partial, end="", flush=True)
    except AttributeError:
        # fallback / final result
        print("\r" + " " * len(buffer), end="")  # 清掉這行
        print("\r" + event.text)  # 印最終結果

## Translations

翻譯 API 接收支持的任何語言的音頻文件作為輸入，並轉錄為英文。這與我們的 /Transcriptions 端點不同，因為輸出不是原始輸入語言的文本，而是轉換為英文文本。

In [ ]:
recorded_frames.clear()
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()
recording_thread.join()

# Combine recorded frames
if recorded_frames:
    audio_np = np.concatenate(recorded_frames, axis=0)
else:
    audio_np = np.array([], dtype=dtype)

mp3_io =  audio_to_mp3_io(audio=audio_np, FS=FS, channels=1)

# 這個接口(end point)不支援 stream
output = client.audio.translations.create(
    model="whisper-1", 
    file=mp3_io,
    response_format="text",
)



In [ ]:
audio_file= open("tutorial/week_6/孩子上網，小心上當.mp3", "rb")

output = client.audio.translations.create(
    model="whisper-1", 
    file=audio_file,
    response_format="text",
)


In [ ]:
output

# Text to Speech

對於智慧型即時應用程式，請使用 gpt-4o-mini-tts 模型——最新且最可靠的文字轉語音（Text-to-Speech, TTS）模型。
你可以透過提示（prompt）來控制語音的多種特性，包括：

>- 口音 (Accent)
>- 情感表現範圍 (Emotional range)
>- 語調 (Intonation)
>- 語氣風格（模仿或印象）(Impressions)
>- 語速 (Speed of speech)
>- 聲音音色 (Tone)
>- 低語（耳語）效果 (Whispering)

Audio API 提供一個基於我們 TTS 模型的語音端點（speech endpoint）。它內建 11 種不同的語音，可用於語音生成與相關應用。

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

# By default, output format is in mp3

speech_file_path = os.path.join("tutorial", "week-6", "Sample.mp3")

response = client.audio.speech.create(
  model="tts-1",
  voice="alloy",
  input="Today is Saturday, how are you?")

# 儲存為檔案
response.stream_to_file(speech_file_path)

# 播放 (simple, blocking)
sound = AudioSegment.from_mp3(speech_file_path)

In [ ]:
sound

gpt-4o-mini-tts 和 gp4-4o-tts 可以透過 prompt來控制語調

In [ ]:
speech_file_path = os.path.join("tutorial/week-6/Sample.mp3")

response = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="nova",
    input="おかえりなさい",
    instructions="Speak in a cute and affection tone, similar to the style of an anime girl.")

response.stream_to_file(speech_file_path)

sound = AudioSegment.from_mp3(speech_file_path)

In [ ]:
sound

In [ ]:
response = client.audio.speech.create(
    model="gpt-4o-mini-tts",
    voice="nova",
    input="おかえりなさい",
    instructions="Speak in a tiresome tone.")

response.stream_to_file(speech_file_path)

sound = AudioSegment.from_mp3(speech_file_path)

In [ ]:
sound

### pygame (good for GUI apps / async playback)

In [ ]:
!pip -q install pygame

In [ ]:
import pygame

pygame.mixer.init()
pygame.mixer.music.load(speech_file_path)
pygame.mixer.music.play()

while pygame.mixer.music.get_busy():
    pygame.time.Clock().tick(10)

### 串流

在整個音訊檔案還沒完全下載完畢之前，播放就已經開始了
將格式從壓縮的mp3換成無壓縮的wav

** 開源社群臥虎藏龍

https://community.openai.com/t/streaming-from-text-to-speech-api/493784/29

In [ ]:
!pip install pyaudio

In [ ]:
from textwrap import dedent

input_ = dedent("""
鳴大鐘一次！
推動杠杆，啟動活塞和泵……
鳴大鐘兩次！
按下按鈕，發動引擎，點燃渦輪，注入生命……
鳴大鐘三次！
齊聲歌唱，讚美萬機之神！
""")

In [ ]:
import os
from time import sleep
import wave
import requests

import pyaudio

# WAV: Uncompressed WAV audio, suitable for low-latency applications to avoid decoding overhead.

url = "https://api.openai.com/v1/audio/speech"
headers = {
    "Authorization": f'Bearer {os.getenv("OPENAI_API_KEY")}',
}

data = {
    "model": "tts-1",
    "input": input_,
    "voice": "shimmer",
    "response_format": "wav",
}

# You send the request to the API (requests.post).
# 這裡的 stream=True 是整個行為的關鍵。
#🔹 stream=True 的作用

# 設定 stream=True 之後，requests 函式庫不會一次把整個伺服器回應內容全部下載下來。
# 它會回傳一個「串流的回應物件」（response.raw），
# 讓你可以邊接收資料、邊處理內容。

# 因此，當 OpenAI API 開始傳送音訊資料時，
# 你就能立即從網路串流中讀取部分音訊，
# 不需要等待整個 WAV 檔案都傳完。
response = requests.post(url, headers=headers, json=data, stream=True)

CHUNK_SIZE = 1024

if response.ok:
    # 這段程式中：
    # wave.open(response.raw, 'rb') 開啟的是一個可「邊讀邊播」的串流。
    with wave.open(response.raw, 'rb') as wf:
        p = pyaudio.PyAudio()
        # You configure the PyAudio stream (p.open(...)).
        stream = p.open(format=p.get_format_from_width(wf.getsampwidth()),
                        channels=wf.getnchannels(),
                        rate=wf.getframerate(),
                        output=True)

        # wf.readframes(CHUNK_SIZE) 每次只讀取一小塊音訊（例如 1024 幀）。
        # 如果下一塊音訊還沒從伺服器傳完，它會短暫等待，然後繼續播放。
        # 因此，音訊一邊從伺服器傳過來，一邊就被播放出去。
        while len(data := wf.readframes(CHUNK_SIZE)):
            #這行負責撥放聲音
            stream.write(data)

        # Sleep to make sure playback has finished before closing
        sleep(1)
        stream.close()
        p.terminate()
else:
    response.raise_for_status()

### 實時串流 (Real time streaming)

語音 API 支援使用「分塊傳輸編碼（chunk transfer encoding）」進行即時音訊串流。
這代表音訊可以在整個檔案尚未完全生成並可存取之前，就開始播放。

為了獲得最快的回應速度，我們建議使用 wav 或 pcm 作為回傳的音訊格式。

In [ ]:
import asyncio

from openai import AsyncOpenAI
from openai.helpers import LocalAudioPlayer

async_openai = AsyncOpenAI()

async def main() -> None:
    async with async_openai.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="coral",
        input="Today is a wonderful day to build something people love!",
        instructions="Speak in a cheerful and positive tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

await main()

# 語音自助點飲料系統

這只是一個DEMO，所以會非常簡陋。我也不清楚這到底有沒有做的價值

假設一個很簡單的使用情境: 一個人講了他對於飲料的需求，然後whisper從語音中抽取他對於飲料的需求:

- 哪種飲料
- 冰
- 糖

先不管利用TTS最回應機制，看看whisper能不能正確抽取內容

### 清心

正式做的話應該是直接開 WebScraping

- 珍珠蜂蜜鮮奶普洱
- 茶凍奶綠
- 嚴選高山茶
- 咖啡奶茶
- 冬瓜檸檬

冰熱: 正常冰 - 少冰 - 微冰 - 去冰
甜度: 無糖 - 微糖 - 半糖 - 少糖

In [ ]:
import os

os.chdir("../../")

把之前學的模板copy/paste

In [ ]:
import os
from textwrap import dedent
from typing import Literal, List

from langchain_openai import ChatOpenAI
from langchain_core.prompts.image import ImagePromptTemplate
from langchain_core.prompts import PromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, SystemMessagePromptTemplate
from langchain_core.runnables import Runnable, chain
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

from initialization import credential_init


def build_standard_chat_prompt_template(kwargs):
    messages = []

    if 'system' in kwargs:
        content = kwargs.get('system')

        # allow list of prompts for multimodal
        if isinstance(content, list):
            prompts = [PromptTemplate(**c) for c in content]
        else:
            prompts = [PromptTemplate(**content)]

        message = SystemMessagePromptTemplate(prompt=prompts)
        messages.append(message)

    if 'human' in kwargs:
        content = kwargs.get('human')

        # allow list of prompts for multimodal
        if isinstance(content, list):
            prompts = []
            for c in content:
                if c.get("type") == "image":
                    prompts.append(ImagePromptTemplate(**c))
                else:
                    prompts.append(PromptTemplate(**c))
        else:
            if content.get("type") == "image":
                prompts = [ImagePromptTemplate(**content)]
            else:
                prompts = [PromptTemplate(**content)]

        message = HumanMessagePromptTemplate(prompt=prompts)
        messages.append(message)

    chat_prompt_template = ChatPromptTemplate.from_messages(messages)
    
    return chat_prompt_template

credential_init()

model = ChatOpenAI(openai_api_key=os.environ['OPENAI_API_KEY'],
                   model_name="gpt-4o-mini", 
                   temperature=0
                  )

### 定義輸出格式

In [ ]:
class Drink(BaseModel):

    name: Literal['珍珠蜂蜜鮮奶普洱', '茶凍奶綠', '嚴選高山茶', 
                  '咖啡奶茶', '冬瓜檸檬'] = Field(None, description="飲料名稱")
    ice_level: Literal['正常冰', '少冰', '微冰', '去冰'] = Field("正常冰", description='冰熱程度')
    sugar_level: Literal['無糖', '微糖', '半糖' , '少糖'] = Field("少糖", description='糖度')


class Order(BaseModel):

    names: List[Drink] = Field(description=("用戶點的飲料"))

output_parser = PydanticOutputParser(pydantic_object=Order)
format_instructions = output_parser.get_format_instructions()

### 模擬從whisper得到用戶需求

In [ ]:
system_template = dedent("""\
You are a friendly and helpful AI assistant acting as a beverage shop clerk (飲料店店員).

Your role is to interact naturally with customers, understand their preferences, and help them place drink orders.

The ordered beverage must be from the 飲料名稱. If not, you will reply `錯誤`
""")

human_template = dedent("""\
                 {query}
                 format instruction: {format_instructions}
                 """)

input_ = {"sytem": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"],
                    "partial_variables": {"format_instructions": 
                                          format_instructions}}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

extraction_pipeline = chat_prompt_template | model | output_parser

## 透過語言模型提取關鍵字

In [ ]:
extraction_pipeline.invoke({"query": "我要一杯冬瓜檸檬和一杯珍珠蜂蜜鮮奶普洱"})

不指定糖度和冰量的話，就會回傳預設值

In [ ]:
extraction_pipeline.invoke({"query": "我要一杯冬瓜檸檬，微糖，去冰"})

In [ ]:
extraction_pipeline.invoke({"query": "我要一杯冬瓜檸檬和一杯嚴選高山茶。冬瓜檸檬微糖去冰，高山茶無糖微冰"})

目前看起來都很正常，萬一用戶要求一個風馬牛不相及的問題呢?

In [ ]:
extraction_pipeline.invoke({"query": "我要一本漫畫"})

Oops...

- PydanticOutputParser 會強制讓輸出結果符合你定義的資料結構（schema）。

- 大型語言模型（LLM）本身並沒有「拒絕回答」或「不適用」的概念 —— 它會盡力讓輸出看起來符合這個結構。

- 因此，即使輸入的內容與飲料訂單完全無關，模型仍可能「幻覺式地」生成一個看似有效的 Order 結果。

In [ ]:
from typing import Optional


class Drink(BaseModel):
    
    name: Literal['珍珠蜂蜜鮮奶普洱', '茶凍奶綠', '嚴選高山茶', 
                  '咖啡奶茶', '冬瓜檸檬'] = Field(..., description="飲料名稱")
    ice_level: Literal['正常冰', '少冰', '微冰', '去冰'] = Field("正常冰", description='冰熱程度')
    sugar_level: Literal['無糖', '微糖', '半糖' , '少糖'] = Field("少糖", description='糖度')


class Order(BaseModel):

    relevant: bool = Field(..., description="是否為飲料訂單。若使用者並非點飲料，請輸出 False。")
    names: Optional[List[Drink]] = Field(default=None, description="若 relevant 為 True，則列出飲料。")


output_parser = PydanticOutputParser(pydantic_object=Order)
format_instructions = output_parser.get_format_instructions()

input_ = {"sytem": {"template": system_template},
          "human": {"template": human_template,
                    "input_variable": ["query"],
                    "partial_variables": {"format_instructions": 
                                          format_instructions}}}

chat_prompt_template = build_standard_chat_prompt_template(input_)

extraction_pipeline = chat_prompt_template | model | output_parser

In [ ]:
extraction_pipeline.invoke({"query": "我要一本漫畫"})


In [ ]:
extraction_pipeline.invoke({"query": "我要一杯啤酒"})


In [ ]:
extraction_pipeline.invoke({"query": "我要一杯冬瓜檸檬和一杯嚴選高山茶。冬瓜檸檬微糖去冰，高山茶無糖微冰"})

透過這個方法，可以避免用戶搞些有的沒的。

另一個選擇是使用Guardrails: 可以使用OpenAI Agent SDK，然後再輸入的地方加入一個檢查系統。

## 輸入從文字換成語音

In [ ]:
import io
import threading

import numpy as np
import sounddevice as sd
from openai import OpenAI
from pydub import AudioSegment

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])

FS = 44100
CHANNELS = 1
dtype = 'int16'

recorded_frames = []

def callback(indata, frames, time, status):
    recorded_frames.append(indata.copy())


def record_audio():
    # 加入起始和結束的控制
    with sd.InputStream(samplerate=FS, channels=CHANNELS, 
                        dtype=dtype, callback=callback):
        input("Press Enter to start recording...")
        print("Recording... Press Enter again to stop.")
        input()
        print("Recording stopped.")


def audio_to_mp3_io(audio, FS:int, channels: int):

    # Convert numpy array to AudioSegment
    audio_bytes = audio.tobytes()
    
    audio_segment = AudioSegment(
        data=audio_bytes,
        sample_width=audio.dtype.itemsize,
        frame_rate=FS,
        channels=channels
    )
    
    # Save as MP3 in-memory (as an object)
    mp3_io = io.BytesIO()
    audio_segment.export(mp3_io, format="mp3")
    mp3_io.seek(0)  # Rewind to start

    mp3_io.name = "word.mp3"

    return mp3_io

我要一杯冬瓜檸檬和一杯嚴選高山茶。冬瓜檸檬微糖去冰，高山茶無糖微冰

In [ ]:
recorded_frames.clear()
recording_thread = threading.Thread(target=record_audio)
recording_thread.start()
recording_thread.join()

In [ ]:
from datetime import datetime


# Combine recorded frames
if recorded_frames:
    audio_np = np.concatenate(recorded_frames, axis=0)
else:
    audio_np = np.array([], dtype=dtype)

mp3_io =  audio_to_mp3_io(audio=audio_np, FS=FS, channels=1)

# 這個接口(end point)不支援 stream

time_begin = datetime.now()
whisper_output = client.audio.transcriptions.create(
    model="whisper-1", 
    file=mp3_io,
    response_format="text",
    prompt="珍珠蜂蜜鮮奶普洱, 茶凍奶綠, 嚴選高山茶, 咖啡奶茶, 冬瓜檸檬, 正常冰, 少冰, 微冰, 去冰, 無糖, 微糖, 半糖, 少糖",
)
time_end = datetime.now()

print(time_end - time_begin)

In [ ]:
whisper_output

## 計算帳單和回應

In [ ]:
orders = extraction_pipeline.invoke({"query": whisper_output})

In [ ]:
price_map = {"珍珠蜂蜜鮮奶普洱": 70,
         "茶凍奶綠": 50,
         "嚴選高山茶": 35,
         "咖啡奶茶": 75,
         "冬瓜檸檬": 60}

In [ ]:
orders.names

In [ ]:
total_price = 0

for order in orders.names:
    total_price += price_map[order.name]

In [ ]:
total_price

使用語音進行最後的回應

In [ ]:
import asyncio

from openai import AsyncOpenAI
from openai.helpers import LocalAudioPlayer

async_openai = AsyncOpenAI()

async def main(price) -> None:
    async with async_openai.audio.speech.with_streaming_response.create(
        model="gpt-4o-mini-tts",
        voice="nova",
        input=f"一共{price}元 💕",
        instructions="Speak in a sweet, energetic, anime-girl style with a cute and playful tone.",
        response_format="pcm",
    ) as response:
        await LocalAudioPlayer().play(response)

await main(price=total_price)

可以的話看能不能找個第三方語音服務供應商，提供那種甜死人不償命的語音服務。生意保證好。

開啟Langserve，進行後端測試

In [ ]:
import json
import requests


whisper_output = '我要一杯冬瓜檸檬和一杯嚴選高山茶, 冬瓜檸檬微糖去冰, 嚴選高山茶無糖微冰。'

payload = {'input': {"query": whisper_output}}

"""
- ensure_ascii=False → keeps Chinese characters as-is.

- .encode('utf-8') → ensures bytes sent are UTF-8.

- Header "charset=utf-8" → tells the server the encoding.
"""

response = requests.post(
        "http://localhost:8080/drinking_app/invoke",
        json=payload
    )

In [ ]:
order = eval(response.text)

In [ ]:
order['output']['names']

In [ ]:
import pandas as pd

price_map = {"珍珠蜂蜜鮮奶普洱": 70,
         "茶凍奶綠": 50,
         "嚴選高山茶": 35,
         "咖啡奶茶": 75,
         "冬瓜檸檬": 60}

df = pd.DataFrame(order['output']['names'])

df['price'] = df['name'].map(price_map)   

In [ ]:
df['price'].sum()

建立起 streamlit + flask + langserve的服務。詳細的內容看 python script檔

# Ollama 概論

Ollama 是一個框架/工具，用於在本地端運行和管理開源大型語言模型（LLM）。它極大地簡化了在個人電腦上部署模型（例如 Llama 2、Mistral 等）的過程。

使用開源的優勢評估:

機敏資料 (Sensitive Data)

>- 當你在本地端使用 Ollama 運行開源模型時，你的資料不會離開你的電腦（除非你主動將模型輸出發送到外部服務）。這與使用商業 API（如 OpenAI、Google Gemini）不同，後者的輸入資料會傳輸到服務提供商的伺服器進行處理。對於處理機密、敏感或受法規管制的資料（例如醫療記錄、專利資訊、商業機密等），本地運行是確保資料隱私和安全的關鍵優勢。

無內容審查 (uncensored)

>- Censorship/內容限制： 商業 API 服務提供商（OpenAI, Google, Anthropic 等）會在其服務條款中嚴格禁止生成特定類型的內容，包括但不限於：非法內容、仇恨言論、暴力內容、露骨的色情內容 (NSFW) 等。這是由於法律、道德和品牌聲譽的考量。

>- 開源模型： 開源社群訓練和發佈的模型有許多是缺乏或設計繞過這些嚴格安全層的（即它們的「護欄」較弱或可調節）。因此，對於需要生成受商業 API 限制的內容（包括 NSFW、爭議性、或繞過倫理限制的內容）的使用者來說，這些**未經審查（uncensored）或調整過（fine-tuned）**的開源模型確實成為一個重要的使用場景。

OpenAI 預告在今年12月，ChatGPT文字上可以產生NFSW內容

In [ ]:
from IPython.display import display, HTML

# Define the HTML to display images side by side
html = """
<div style="display: flex; justify-content: space-around;">
    <div>
        <img src="Sam Altman.png" height="900" width="600" />
    </div>
</div>
"""

# Display the HTML
display(HTML(html))

This package enables you using open-source LLM with ease.

## Linux系統上安裝

https://medium.com/@abonia/running-ollama-in-google-colab-free-tier-545609258453

- curl https://ollama.ai/install.sh | sh
- ollama serve &
- ollama pull llama3:8b
- ollama pull dolphin-llama3:8b
- ollama pull huihui_ai/qwen2.5-abliterate:14b

** 還沒真實測是過

## Windows

- 下載 & 安裝
- pip install ollama

In [ ]:
!pip install -U ollama

## 我們下載了兩個Ollama，差異在哪裡?

### 🖥️ **ollama.exe（獨立安裝程式 / 下載版）**

當你從官方的 [Ollama 網站](https://ollama.com/download) 下載 **`ollama.exe`** 時，  
你得到的是 **Ollama 的核心執行環境** —— 也就是實際在本機上執行模型的引擎。

✅ 內容包含：

- Ollama 伺服器（背景執行程序）  
- Ollama 命令列介面（CLI）  
- GPU / CPU 整合  
- 本地模型管理（下載、快取、提供模型服務）

💡 若你想在自己的電腦上實際執行模型，這個版本是 **必要的**。

---

### 🐍 **pip install ollama（Python 套件）**

Python 套件 **不包含執行環境** —— 它只是用來與已啟動的 Ollama 伺服器互動的 **用戶端函式庫**。

✅ 它能讓你：

- 向本地的 Ollama API（`http://localhost:11434`）發送請求  
- 從 Python 執行並串流模型回應  
- 在 Python 腳本或 Notebook 中使用 Ollama


In [ ]:
import os

os.chdir("../../")

## Cloud API Access

In [ ]:
from ollama import Client

from initialization import credential_init

credential_init()

client = Client(
    host="https://ollama.com",
    headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')}
)

messages = [
  {
    'role': 'user',
    'content': '天空為何是藍色的?',
  },
]

for part in client.chat('gpt-oss:120b', messages=messages, stream=True):
    print(part['message']['content'], end='', flush=True)


# 下載模型
# client.pull('gpt-oss:20b')

## 搭配 Langchain 使用

In [ ]:
from langchain_community.chat_models.ollama import ChatOllama

model_ollama = ChatOllama(model='gpt-oss:120b', base_url='https://ollama.com',
                          headers={'Authorization': 'Bearer ' + os.environ.get('OLLAMA_API_KEY')})


In [ ]:
model_ollama.invoke("天空為何是藍色的?")

## 下載模型

建議直接在CLI介面使用

接下來我們會在Colab上看如何使用本地模型